In [1]:
import pandas as pd
import numpy as np

import os
import json
import datetime

from matplotlib import pyplot as plt
from scipy import stats, special

import utils
import links
import generators
import estimation

%load_ext autoreload
%autoreload 2



# Definitions of non-intuitive things

#### What is X?
X is a matrix of weather conditions generated in 1-InputGeneration.ipynb

The data are synthetic, but they're based on statistical properties of real weather data.

#### What is a scenario?
Different "scenarios" represent different failure processes used to generate the data. These could be constant, piecewise linear, etc.

Sceanrios are pre-defined in "inputs/scenarios.csv". Right now they're completely made up. Next steps are to use realistic assumptions about the lifetime and failure properties of grid assets (ultimately this piece goes in 2-ScenarioGeneration.ipynb - but for now that notebook is a mess and accomplishes nothing)



In [149]:
fleet_size=10000
X = pd.read_csv('inputs/weather.csv', index_col='time')
X.head()

scenarios = pd.read_csv('inputs/scenarios.csv', index_col='name')



In [150]:

        
for i, scenario in scenarios.iterrows():
    if i != 28:
        continue
    params = generators.params(scenario, fleet_size=fleet_size)
    failures, failure_rate = generators.failure_data(params, X, links.Link, fleet_size=fleet_size)
    if 'scenario%i'%(i) not in os.listdir('scenarios/'):
        os.mkdir(os.path.join('scenarios', 'scenario%i'%(i)))
        os.mkdir(os.path.join('scenarios', 'scenario%i'%(i), 'plots'))
        
    df = pd.DataFrame(index=failures.index)
    df['count'] = failures
    df['rate'] = failure_rate
    
    print failures.max()

    df.to_csv(os.path.join('scenarios','scenario%i'%(i),'failures.csv'), header=True)
    params.to_csv(os.path.join('scenarios','scenario%i'%(i),'parameters.csv'), header=True)

    

191.0


# Scenario analysis
It takes quite a while to run through a scenario, let alone all of them. If you're just playing around, you can change "scenarios_list" to run only a subset of scenarios, or change "mcmc_stopping_criteria" to stop after fewer iterations. People will run ~1e5 or 1e6 iterations, but if the model is a good fit (big if, see 5.2) then it converges much faster.

Just a heads up. :-)

In [152]:

# USE RANDOM WALK
mcmc_random_walk = lambda theta, cov: pd.Series(stats.multivariate_normal.rvs(mean=theta, cov=cov),
                                           index=theta.index)

scenarios = pd.read_csv('inputs/scenarios.csv', index_col='name')

fleet_size = 10000
scenarios_list = scenarios.index
mcmc_stopping_criteria = lambda it: it<5e4

for i in scenarios_list:
    if i != 19:
        continue
    scenario = 'scenario%i'%(i)

    y = pd.read_csv(os.path.join('scenarios', scenario, 'failures.csv'), index_col='time')['count']
    
    if y.sum() == 0:
        continue

#     x_vars = [['Wind',],['Wind','Precip',],['Wind','Precip','WindStorm'],]
    x_vars = [['Wind',],['Wind','DayPrecip',],['Wind','DayPrecip','WindStorm'],]
    for var in x_vars:
        model_name = '+'.join(var)
        
        if os.path.exists(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name))):
            continue
            
        _X = X[var]

        # -------------------------------
        # TUNING MCMC SAMPLER
        # Transition in the MCMC chain is a random walk where step size is gaussian and spherical
        # If the step size is too big, parameter updates are rarely accepted (poor mixing)
        # If the step size is too small, autocorrelation is high, convergence is slow
        # in both cases the chain needs to be longer.
        # Rule of thumb is to choose sigma so that acceptance rate is 25%
        
        
#         cov, p0 = estimation.transition_function(_X, y, links.Link(), fleet_size, tol=0.01, chain_size=1000)
#         estimation.save_chain_params(cov, p0, scenario, model_name)
        
#         out = estimation.get_existing_transition_params(scenario, model_name)
        out = []
        if len(out) == 0:
            cov, p0 = estimation.transition_function(_X, y, links.Link(), fleet_size, tol=0.01, chain_size=1000)
            estimation.save_chain_params(cov, p0, scenario, model_name)
        else:
            cov, p0 = out
            
        params, acceptance = estimation.metropolis(p0, _X, y, links.Link(), fleet_size, 
                                                   jump=mcmc_random_walk, accept_reject=True, 
                                                   cov=cov, stop=mcmc_stopping_criteria)
        params['acceptance'] = acceptance

        if not os.path.exists(os.path.join('scenarios', scenario, 'chains')):
            os.mkdir(os.path.join('scenarios', scenario, 'chains'))

        params.to_csv(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name)))
        
        
        

ValueError: cannot reshape array of size 0 into shape (0)

In [153]:
y.sum()

14673.0

In [103]:
scenario


'scenario27'

In [64]:
cov, theta_star

NameError: name 'theta_star' is not defined

In [65]:
p0

slope.Precip       -10.070734
slope.Wind           0.340613
threshold.Precip     0.000000
threshold.Wind      46.502160
dtype: float64